# Time Series Analysis

## Dependencies and helper methods

## First evolution of the lake

In [ ]:
# Display area and detect images with abrupt change in segment with respect to the the previous and the next one 
array_area = np.array([np.sum(segment) for index, segment in enumerate(array_segmented_images)])

# Get time in proper units
array_times_int = np.array( [(x - datetime.datetime(1970, 1, 1)).total_seconds() for x in array_times])
array_times_int = (array_times_int - array_times_int[0]) / 10**6

# Interpolate the signal with a regular sampling for smoothing afterwards
f = interp1d(array_times_int, array_area, kind='linear')
xnew = np.linspace(0, array_times_int[-1], num=500, endpoint=True)
y_new = f(xnew)

# Smooth the signal with a Savgol filter
yhat = savgol_filter(y_new, 7, 3) # window size 7, polynomial order 3

# Reinterpolate 
f = interp1d(xnew, yhat, kind='linear')
y_hat_proper_sampling = f(array_times_int)


# Plot the smoothed signal
plt.figure(figsize=(10,5),facecolor="white")
plt.plot(array_times, array_area * 100/1000000, 'o', array_times, y_hat_proper_sampling* 100/1000000, '-')
plt.xlabel('Time')
plt.ylabel('Area (km^2)')
plt.title('Evolution of lake average area with time')
plt.show()

In [ ]:
# Plot histogram of relative error compared to smoothed signal 
fig, ax = plt.subplots(1, figsize = (10,5))
l_relative_error = (array_area-y_hat_proper_sampling)**2/np.array(y_hat_proper_sampling+1)
fig.patch.set_facecolor('white')
plt.hist(l_relative_error , bins=50)
plt.xlim(0, 15000)
plt.xlabel("Error")
plt.ylabel("Frequency")
tresh = 1700
# Create one rectangle patch and add it to the plot
rect = patches.Rectangle((tresh, 0), 15000-tresh, 150, alpha = 0.3, facecolor="red")
ax.add_patch(rect)

ax.set_yscale('log')
#plt.ylim(0,150)
plt.title('Relative error of area compared to smoothed signal')
plt.show()

## Study of the algae content within segmented regions

In [ ]:
# Compute algae content for each acquisition
l_mean_algae = [np.mean(array_data_filtered[index,:,:,6][segment==1]) if np.sum(segment)>0 else 0 for index, segment in enumerate(array_segmented_images_filtered)]

# Smooth the signal with a Savgol filter
f = interp1d(array_times_int_filtered, l_mean_algae, kind='slinear')
xnew = np.linspace(0, array_times_int_filtered[-1], num=500, endpoint=True)
yhat_algae = [x if x>=0 else 0 for x  in savgol_filter(f(xnew), 21, 3)]
x_times = [datetime.datetime.fromtimestamp(x) for x in xnew * 10**6 + array_times_filtered[0].timestamp() ]

# Plot the final curve
fig, ax = plt.subplots(figsize=(10,5))
fig.patch.set_facecolor('white')
plt.plot(array_times_filtered , l_mean_algae, '.', c = 'C2')
plt.plot( x_times, yhat_algae, '-', c = 'C0')

plt.xlabel('Time')
plt.ylabel('Algae content')
plt.xlim(array_times_filtered[0], array_times_filtered[-1])
#plt.ylim([0, np.max(array_area_filtered+2000)])
ax.xaxis.set_major_locator(mdates.MonthLocator(interval=6))
plt.title('Evolution of lake algae content with time')
plt.show()

In [ ]:
# Scale for main axes
ylims = [0, np.max(array_area_filtered+2)]
xlims = mdates.date2num([array_times_filtered[0], array_times_filtered[-1]])

# Scale for the colorbar
min_algae = np.min(yhat_algae)
max_algae = np.max(yhat_algae)
scale_algae = max_algae - min_algae

# Draw the image over the whole plot area
fig, ax = plt.subplots(figsize=(10,5))
fig.patch.set_facecolor('white')
im = plt.imshow(np.array([yhat_algae]), cmap='crest_r', origin='lower', extent=[xlims[0], xlims[1], ylims[0],  ylims[1]],aspect='auto')

# Erase above the data by filling with white
plt.fill_between(x_times, yhat, ylims[1], color='w')

# Make the line plot over the top
plt.plot(x_times, yhat, 'C0', linewidth=0.5)


# Set limits
ax.set_xlim(xlims)
ax.set_ylim(ylims)

# Labels
plt.xlabel('Time')
plt.ylabel('Lake area ' + r'($km^2$)')
ax.set_title('Evolution of lake area and algae composition over time', y=1.0, pad=-18)

# More customization
ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)


# Define the date format
ax.xaxis.set_major_locator(mdates.MonthLocator(interval=6))

#add color bar
cbaxes = inset_axes(ax, width="2%", height="50%", bbox_to_anchor = [0,0,0.85,0.8], bbox_transform=ax.transAxes, loc=1) 
t = fig.colorbar(im, cax=cbaxes, orientation='vertical', ticks=[min_algae + scale_algae * 0.05, max_algae - scale_algae * 0.05])
cbaxes.set_title('Algae content', size=10)
cbaxes.set_yticklabels(['Low algae', 'High algae'], size = 10) 
t.outline.set_visible(False)

plt.show()